In [24]:
import pandas as pd                       #to perform data manipulation and analysis
import numpy as np                        #to cleanse data
from datetime import datetime             #to manipulate dates
import plotly.express as px               #to create interactive charts
import plotly.graph_objects as go         #to create interactive charts
from jupyter_dash import JupyterDash      #to build Dash apps from Jupyter environments
from dash import dcc        #to get components for interactive user interfaces
from dash import html       #to compose the dash layout using Python structures
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import datetime
from dash import dash_table
import json
import calendar

import init_database

In [4]:
# read constant
with open('data.json','r') as f:
    data = json.load(f)
    NECESSITY_MONEY = data['constant']['MONEY_ALLOCATED_FOR_NECESSITY_PER_MONTH']
    EDUCATION_AND_ENTERTAINMET = data['constant']['MONEY_ALLOCATED_FOR_EDUCATION_AND_ENTERTAINMENT_PER_MONTH']
    

In [3]:
# read info page
with open('info_page.md', 'r') as f:
    info_content = f.read()

In [5]:
# declare today
today_date = datetime.datetime.now().strftime("%d").zfill(2)
today_month = datetime.datetime.now().strftime("%m").zfill(2)
today_month_EN = datetime.datetime.now().strftime("%B")
today_year = datetime.datetime.now().strftime("%Y")

In [6]:
# !bash bal_to_csv.sh "01/01" "11/01"
# df = pd.read_csv(database_name)

# # remove '?' column
# df.drop(columns=['?'], inplace=True)
# df.drop(columns=['pending/cleared'], inplace=True)


# # seprarate account into categories
# transaction_type = []
# category = []
# sub_category = [] 

# for i in df['account']:
#     transaction_type.append(i.split(':')[0])    
#     category.append(i.split(':')[1])
#     try:
#         sub_category.append(i.split(':')[2])
#     except:
#         print(i)

# df['transaction_type'] = transaction_type
# df['category'] = category
# df['sub_category'] = sub_category

In [8]:
# declare first date in the df
first_day_df = pd.to_datetime(df['date']).min()
first_day_date = first_day_df.strftime("%d")
first_day_month = first_day_df.strftime("%m")
first_day_year = first_day_df.strftime("%Y")
first_day_df = first_day_df.strftime("%Y-%m-%d")

In [25]:
df = init_database.init_database()

In [26]:
df['sub_category'].unique()
df['date_of_week'].unique()
df['category'].unique()

array(['Wallet', 'Card', 'eWallet', 'Balance', 'Food', 'Travel', 'Drink',
       'Fulbright', 'Transportation', 'Education', 'Personal',
       'Household Items', 'Part-time Jobs', 'Fees', 'Utility', 'Home',
       'Entertainment', 'Fintech', 'Givings', 'Health', 'Internship',
       'All'], dtype=object)

In [11]:
def money_in_visa_card():
    return df[df['sub_category'] == 'VISA']['amount'].sum()

def money_in_acb_card():
    return df[df['sub_category'] == 'ACB']['amount'].sum()

def money_in_finhay():
    return df[df['sub_category'] == 'Finhay']['amount'].sum()

def money_in_cash():
    return df[df['sub_category'] == 'Cash']['amount'].sum()

def money_in_home():
    return df[df['sub_category'] == 'Home']['amount'].sum()

def money_in_moca():
    return df[df['sub_category'] == 'MOCA']['amount'].sum()

In [27]:
transaction_list = df['transaction_type'].unique().tolist()
transaction_list = ['All'] + transaction_list

transaction_sub_category = df['sub_category'].unique().tolist()
transaction_sub_category = ['All'] + transaction_sub_category

transaction_list

['All', 'Assets', 'Equity', 'Expenses', 'Income', 'Saving']

In [22]:
# Build App
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.Div([
        dcc.Tabs([
# --------------------------------------- Overview tab ---------------------------------------

            dcc.Tab(label='Overview', children=[
                
                # ----------------------- Graph 01 -----------------------
                html.Div([
                    html.H2(f"Expenses for Necessity in {calendar.month_name[int(today_month)]}"),
                                
                    dcc.Graph(figure=create_expenses_by_category(df, ['Food', 
                                                                      'Drink', 
                                                                      'Transportation', 
                                                                      'Personal', 
                                                                      'Household Items',
                                                                      'Fees'], today_month, today_year))
            
                ], style={'width': '48%', 'display': 'inline-block'}),
                
                # ----------------------- Graph 02 -----------------------
                
                html.Div([
                    html.H2(f"Expenses for Entertainment & Education in {calendar.month_name[int(today_month)]}"),
                    
                    dcc.Graph(figure=create_expenses_by_category(df, ['Education',
                                                                      'Entertainment'], today_month, today_year))
                    
                ], style={'width': '48%', 'display': 'inline-block'}),
                
                # ----------------------- Graph 03 -----------------------
                html.Div([
                    html.H2("Monthly Expenses"),
                                
                    dcc.Graph(figure=create_plot_for_category(df))
            
                ], style={'width': '48%', 'display': 'inline-block'}),
                
                # ----------------------- Graph 04 -----------------------
                html.Div([
                    html.H2("Expenses by Category"),
                    
                    # ---------
                    html.Div([
                        html.Div("From", 
                                 style={'width': '10%', 
                                        'height':'50%',
                                        'display': 'block',
#                                   'border': '3px solid green',
                                 }),
                        
                        html.Div([
                            dcc.Dropdown(
                                id="start-date-pie",
                                options=[{"label": str(i).zfill(2), "value": str(i).zfill(2)} for i in range(1, 32)],
                                value=first_day_date.zfill(2),
                                clearable=False
                            ),
                        ], style={'width': '15%', 'display': 'inline-block'}),

                        html.Div([
                            dcc.Dropdown(
                                id="start-month-pie",
                                options=[{"label": 'January'  , "value": '01'},
                                         {"label": 'February' , "value": '02'},
                                         {"label": 'March'    , "value": '03'},
                                         {"label": 'April'    , "value": '04'},
                                         {"label": 'May'      , "value": '05'},
                                         {"label": 'June'     , "value": '06'},
                                         {"label": 'July'     , "value": '07'},
                                         {"label": 'August'   , "value": '08'},
                                         {"label": 'September', "value": '09'},
                                         {"label": 'October'  , "value": '10'},
                                         {"label": 'November' , "value": '11'},
                                         {"label": 'December' , "value": '12'}],
                                value=first_day_month,
                                clearable=False
                            ),
                        ], style={'width': '30%', 'display': 'inline-block'}),

                        html.Div([
                            dcc.Dropdown(
                                id="start-year-pie",
                                options=[{"label": i  , "value": i} for i in range(int(first_day_year), datetime.datetime.now().year+1)],
                                value=first_day_year,
                                clearable=False
                            ),
                        ], style={'width': '25%', 'display': 'inline-block'})
                    ], style={'width': '50%', 'display': 'inline-block'}),
                    #----------
                    
                    html.Div([
                        html.Div("To", 
                                 style={'width': '10%', 
                                        'height':'50%',
                                        'display': 'block',
                                 }),

                        html.Div([
                            dcc.Dropdown(
                                id="end-date-pie",
                                options=[{"label": str(i).zfill(2), "value": str(i).zfill(2)} for i in range(1, 32)],
                                value=today_date,
                                clearable=False
                            ),
                        ], style={'width': '15%', 'display': 'inline-block'}),

                        html.Div([
                            dcc.Dropdown(
                                id="end-month-pie",
                                options=[{"label": 'January'  , "value": '01'},
                                         {"label": 'February' , "value": '02'},
                                         {"label": 'March'    , "value": '03'},
                                         {"label": 'April'    , "value": '04'},
                                         {"label": 'May'      , "value": '05'},
                                         {"label": 'June'     , "value": '06'},
                                         {"label": 'July'     , "value": '07'},
                                         {"label": 'August'   , "value": '08'},
                                         {"label": 'September', "value": '09'},
                                         {"label": 'October'  , "value": '10'},
                                         {"label": 'November' , "value": '11'},
                                         {"label": 'December' , "value": '12'}],
                                value=today_month,
                                clearable=False
                            ),
                        ], style={'width': '30%', 'display': 'inline-block'}),

                        html.Div([
                            dcc.Dropdown(
                                id="end-year-pie",
                                options=[{"label": i  , "value": i} for i in range(int(first_day_year), datetime.datetime.now().year+1)],
                                value=today_year,
                                clearable=False
                            ),
                        ], style={'width': '25%', 'display': 'inline-block'})
                    ], style={'width': '50%', 'display': 'inline-block'}),
                        
                    dcc.Graph(id='category-pie')
                    
                ], style={'width': '48%', 'display': 'inline-block'}),
            ]),
            
# --------------------------------------- Transaction tab ---------------------------------------
            dcc.Tab(label='Transactions', children=[
                html.Div([
                    html.H1("Culmulative Expenses in Month"),
                    html.Div([
                        dcc.Dropdown(
                            id="month_cumulative",
                            options=[{"label": 'January'  , "value": '01'},
                                     {"label": 'February' , "value": '02'},
                                     {"label": 'March'    , "value": '03'},
                                     {"label": 'April'    , "value": '04'},
                                     {"label": 'May'      , "value": '05'},
                                     {"label": 'June'     , "value": '06'},
                                     {"label": 'July'     , "value": '07'},
                                     {"label": 'August'   , "value": '08'},
                                     {"label": 'September', "value": '09'},
                                     {"label": 'October'  , "value": '10'},
                                     {"label": 'November' , "value": '11'},
                                     {"label": 'December' , "value": '12'}],
                            value=today_month,
                            clearable=False
                        ),
                    ], style={'width': '18%', 'display': 'inline-block'}),

                    html.Div([
                        dcc.Dropdown(
                            id="year_cumulative",
                            options=[{"label": i, "value": i} for i in range(int(first_day_year), datetime.datetime.now().year+1)],
                            value=today_year,
                            clearable=False
                        ),
                    ], style={'width': '10%', 'display': 'inline-block'}),             

                    dcc.Graph(id='cumulative_monthly_expenses')
               ], style={'width': '48%', 'display': 'inline-block'}),
                
                html.Div([
                    html.H1("Daily Expenses in Month"),
                        html.Div([
                            dcc.Dropdown(
                                id="month_daily",
                                options=[{"label": 'January'  , "value": '01'},
                                         {"label": 'February' , "value": '02'},
                                         {"label": 'March'    , "value": '03'},
                                         {"label": 'April'    , "value": '04'},
                                         {"label": 'May'      , "value": '05'},
                                         {"label": 'June'     , "value": '06'},
                                         {"label": 'July'     , "value": '07'},
                                         {"label": 'August'   , "value": '08'},
                                         {"label": 'September', "value": '09'},
                                         {"label": 'October'  , "value": '10'},
                                         {"label": 'November' , "value": '11'},
                                         {"label": 'December' , "value": '12'}],
                                value=today_month,
                                clearable=False
                            ),
                        ], style={'width': '18%', 'display': 'inline-block'}),

                        html.Div([
                            dcc.Dropdown(
                                id="year_daily",
                                options=[{"label": i, "value": i} for i in range(int(first_day_year), datetime.datetime.now().year+1)],
                                value=today_year,
                                clearable=False
                            ),
                        ], style={'width': '10%', 'display': 'inline-block'}),             

                        dcc.Graph(id='daily_expenses')
                ], style={'width': '48%', 'display': 'inline-block'}),
                
                html.Div([
                    html.H1("Transaction History"),
                    
                    # ---------
                    html.Div([
                        html.Div("From", 
                                 style={'width': '10%', 
                                        'height':'50%',
                                        'display': 'block',
                                 }),

                        html.Div([
                            dcc.Dropdown(
                                id="start-date-transaction",
                                options=[{"label": str(i).zfill(2), "value": str(i).zfill(2)} for i in range(1, 32)],
                                value=today_date.zfill(2),
                                clearable=False
                            ),
                        ], style={'width': '20%', 'display': 'inline-block'}),

                        html.Div([
                            dcc.Dropdown(
                                id="start-month-transaction",
                                options=[{"label": 'January'  , "value": '01'},
                                         {"label": 'February' , "value": '02'},
                                         {"label": 'March'    , "value": '03'},
                                         {"label": 'April'    , "value": '04'},
                                         {"label": 'May'      , "value": '05'},
                                         {"label": 'June'     , "value": '06'},
                                         {"label": 'July'     , "value": '07'},
                                         {"label": 'August'   , "value": '08'},
                                         {"label": 'September', "value": '09'},
                                         {"label": 'October'  , "value": '10'},
                                         {"label": 'November' , "value": '11'},
                                         {"label": 'December' , "value": '12'}],
                                value=today_month,
                                clearable=False
                            ),
                        ], style={'width': '35%', 'display': 'inline-block'}),

                        html.Div([
                            dcc.Dropdown(
                                id="start-year-transaction",
                                options=[{"label": i  , "value": i} for i in range(int(first_day_year), datetime.datetime.now().year+1)],
                                value=today_year,
                                clearable=False
                            ),
                        ], style={'width': '20%', 'display': 'inline-block'})
                    ], style={'width': '25%', 'display': 'inline-block'}),
                    #----------
                    
                    html.Div([
                        html.Div("To", 
                                     style={'width': '10%', 
                                            'height':'50%',
                                            'display': 'block',
                                     }),
                        html.Div([
                            dcc.Dropdown(
                                id="end-date-transaction",
                                options=[{"label": str(i).zfill(2), "value": str(i).zfill(2)} for i in range(1, 32)],
                                value=today_date,
                                clearable=False
                            ),
                        ], style={'width': '10%', 'display': 'inline-block'}),

                        html.Div([
                            dcc.Dropdown(
                                id="end-month-transaction",
                                options=[{"label": 'January'  , "value": '01'},
                                         {"label": 'February' , "value": '02'},
                                         {"label": 'March'    , "value": '03'},
                                         {"label": 'April'    , "value": '04'},
                                         {"label": 'May'      , "value": '05'},
                                         {"label": 'June'     , "value": '06'},
                                         {"label": 'July'     , "value": '07'},
                                         {"label": 'August'   , "value": '08'},
                                         {"label": 'September', "value": '09'},
                                         {"label": 'October'  , "value": '10'},
                                         {"label": 'November' , "value": '11'},
                                         {"label": 'December' , "value": '12'}],
                                value=today_month,
                                clearable=False
                            ),
                        ], style={'width': '18%', 'display': 'inline-block'}),
                        
                        html.Div([
                            dcc.Dropdown(
                                id="end-year-transaction",
                                options=[{"label": i  , "value": i} for i in range(int(first_day_year), datetime.datetime.now().year+1)],
                                value=first_day_year,
                                clearable=False
                            ),
                        ], style={'width': '10%', 'display': 'inline-block'})
                    ], style={'width': '50%', 'display': 'inline-block'}),
                    # ---------             

                    html.Div([
                        html.Div([
                            html.Div("Type", 
                                 style={'width': '10%', 
                                        'height':'50%',
                                        'display': 'block',
                                 }),
                            
                            dcc.Dropdown(
                                id="transaction-type-filter",
                                options=[{"label": i, "value": i} for i in transaction_list],
                                value='Expenses',
                                clearable=False
                            ),
                        ], style={'width': '8%', 'display': 'inline-block'}),

                        html.Div([
                            html.Div("Category", 
                                 style={'width': '10%', 
                                        'height':'50%',
                                        'display': 'block',
                                 }),
                            
                            dcc.Dropdown(
                                id="transaction-category-filter",
                                value='All',
                                clearable=False
                            ),
                        ], style={'width': '11%', 'display': 'inline-block'}),

                        html.Div([
                            
                            html.Div("Sub-category", 
                                 style={
                                        'display': 'block',
                                 }),
                            
                            dcc.Dropdown(
                                id="transaction-sub-category-filter",
                                value='All',
                                clearable=False
                            ),
                        ], style={'width': '12%', 'display': 'inline-block'})
                    ], style={'display': 'block'}),
                    
                    # --------
                    
                    dash_table.DataTable(
                            id='transaction-history',
                            data=df[display_columns].to_dict('records'),
                            columns=[{"name": i, "id": i} for i in display_columns],
                            export_format='xlsx',
                            export_headers='display',
                            merge_duplicate_headers=True,
                            fixed_rows={'headers': True},
                            style_table={'height': 400},
                            style_cell={
                                'minWidth': 80, 'maxWidth': 250, 'width': 95
                            }
                       ),

                   
                ]),

                html.Br(),    
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                
            
            ]),
            
            
            
            dcc.Tab(label='Accounts', children=[
                html.Div([
                    html.H2("Wallets"),
                    # Cash
                    html.Div([
                        # image
                        html.Div([
                            html.Img(
                                src="/assets/100K_cash.jpeg",
                                height=30
                            )
                        ], style={'display':'inline-block',
                                  "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)",
                                  "border-radius": "5px 5px 5px 5px",
                                  "height": "30px"}),
                        
                        # amount
                        html.Div([
                            html.P(f"VND {money_in_cash():,}")
                        ], style={'display':'inline-block', 
                                  "height": "30px", 
                                  "text-align": "center",
                                  "float":"right",
                                 }),
                        
                        html.Hr(
                            className="dashed"
                        ),
                        
                        html.Div([
                            html.H5("Cash")
                        ]),
                        
                    ], style={"width":"200px",
                              "height":"80px",
                              "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)", 
                              "border-radius": "5px",
                              "padding": "20px 20px",
                              'display':'inline-block',
                              "margin-right": "20px",
                              "margin-bottom": "20px"
                              }),
                    
                    # Home savings
                    html.Div([
                        # image
                        html.Div([
                            html.Img(
                                src="/assets/100K_cash.jpeg",
                                height=30
                            )
                        ], style={'display':'inline-block',
                                  "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)",
                                  "border-radius": "5px 5px 5px 5px",
                                  "height": "30px"}),
                        
                        # amount
                        html.Div([
                            html.P(f"VND {money_in_home():,}")
                        ], style={'display':'inline-block', 
                                  "height": "30px", 
                                  "text-align": "center",
                                  "float":"right",
                                 }),
                        
                        html.Hr(
                            className="dashed"
                        ),
                        
                        html.Div([
                            html.H5("Cash at Home")
                        ]),
                        
                    ], style={"width":"200px",
                              "height":"80px",
                              "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)", 
                              "border-radius": "5px",
                              "padding": "20px 20px",
                              'display':'inline-block',
                              "margin-right": "20px",
                              "margin-bottom": "20px"
                             }),
                    
        
                    html.Hr(className="dashed")
                ]),
                
                html.Div([
                    html.H2("eWallets"),
                    
                    # MOCA
                    html.Div([
                        # image
                        html.Div([
                            html.Img(
                                src="/assets/moca.jpeg",
                                height=30
                            )
                        ], style={'display':'inline-block',
                                  "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)",
                                  "border-radius": "5px 5px 5px 5px",
                                  "height": "30px"}),
                        
                        # amount
                        html.Div([
                            html.P(f"VND {money_in_moca():,}")
                        ], style={'display':'inline-block', 
                                  "height": "30px", 
                                  "text-align": "center",
                                  "float":"right",
                                 }),
                        
                        html.Hr(
                            className="dashed"
                        ),
                        
                        html.Div([
                            html.H5("MOCA")
                        ]),
                        
                    ], style={"width":"200px",
                              "height":"80px",
                              "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)", 
                              "border-radius": "5px",
                              "padding": "20px 20px",
                              'display':'inline-block',
                              "margin-right": "20px",
                              "margin-bottom": "20px"
                              }),
                    
                    html.Hr(className="dashed")
                ]),
                
                html.Div([
                    html.H2("Plastic Cards"),
                    
                    # ACB
                    html.Div([
                        # image
                        html.Div([
                            html.Img(
                                src="/assets/acb_card.png",
                                height=30
                            )
                        ], style={'display':'inline-block',
                                  "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)",
                                  "border-radius": "5px 5px 5px 5px",
                                  "height": "30px"}),
                        
                        # amount
                        html.Div([
                            html.P(f"VND {money_in_acb_card():,}")
                        ], style={'display':'inline-block', 
                                  "height": "30px", 
                                  "text-align": "center",
                                  "float":"right",
                                 }),
                        
                        html.Hr(
                            className="dashed"
                        ),
                        
                        html.Div([
                            html.H5("Fulbright ACB")
                        ]),
                        
                    ], style={"width":"200px",
                              "height":"80px",
                              "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)", 
                              "border-radius": "5px",
                              "padding": "20px 20px",
                              'display':'inline-block',
                              "margin-right": "20px",
                              "margin-bottom": "20px"
                             }),
                    
                    # VCB VISA
                    html.Div([
                        # image
                        html.Div([
                            html.Img(
                                src="/assets/visa_card.jpeg",
                                height=30
                            )
                        ], style={'display':'inline-block',
                                  "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)",
                                  "border-radius": "5px 5px 5px 5px",
                                  "height": "30px"}),
                        
                        # amount
                        html.Div([
                            html.P(f"VND {money_in_visa_card():,}")
                        ], style={'display':'inline-block', 
                                  "height": "30px", 
                                  "text-align": "center",
                                  "float":"right",
                                 }),
                        
                        html.Hr(
                            className="dashed"
                        ),
                        
                        html.Div([
                            html.H5("VISA Card")
                        ]),
                        
                    ], style={"width":"200px",
                              "height":"80px",
                              "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)", 
                              "border-radius": "5px",
                              "padding": "20px 20px",
                              'display':'inline-block',
                              "margin-right": "20px",
                              "margin-bottom": "20px"
                             }),
    
                    html.Hr(className="dashed")
                ]), 
                
                
                html.Div([
                    html.H2("Savings"),
                    
                    # Finhay Profit
                    html.Div([
                        # image
                        html.Div([
                            html.Img(
                                src="/assets/finhay.png",
                                height=30
                            )
                        ], style={'display':'inline-block',
                                  "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)",
                                  "border-radius": "5px 5px 5px 5px",
                                  "height": "30px",
                                  "margin-right": "20px"
                                 }),
                        
                        # amount
                        html.Div([
                            html.P(f"VND {money_in_finhay():,}")
                        ], style={'display':'inline-block', 
                                  "height": "30px", 
                                  "text-align": "center",
                                  "float":"right",
                                 }),
                        
                        html.Hr(
                            className="dashed"
                        ),
                        
                        html.Div([
                            html.H5("Finhay Profits")
                        ]),
                        
                    ], style={"width":"200px",
                              "height":"80px",
                              "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)", 
                              "border-radius": "5px",
                              "padding": "20px 20px",
                              'display':'inline-block',
                              "margin-right": "20px",
                              "margin-bottom": "20px"
                              }),
                    
                    # 3-month savings
                    html.Div([
                        # image
                        html.Div([
                            html.Img(
                                src="/assets/finhay.png",
                                height=30
                            )
                        ], style={'display':'inline-block',
                                  "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)",
                                  "border-radius": "5px 5px 5px 5px",
                                  "height": "30px"}),
                        
                        # amount
                        html.Div([
                            html.P(f"VND {money_in_finhay():,}")
                        ], style={'display':'inline-block', 
                                  "height": "30px", 
                                  "text-align": "center",
                                  "float":"right",
                                 }),
                        
                        html.Hr(
                            className="dashed"
                        ),
                        
                        html.Div([
                            html.H5("3-Month Savings")
                        ]),
                        
                    ], style={"width":"200px",
                              "height":"80px",
                              "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)", 
                              "border-radius": "5px",
                              "padding": "20px 20px",
                              'display':'inline-block',
                              "margin-right": "20px",
                              "margin-bottom": "20px"
                              }),
                    
                    # Finhay Gold
                    html.Div([
                        # image
                        html.Div([
                            html.Img(
                                src="/assets/finhay.png",
                                height=30
                            )
                        ], style={'display':'inline-block',
                                  "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)",
                                  "border-radius": "5px 5px 5px 5px",
                                  "height": "30px"}),
                        
                        # amount
                        html.Div([
                            html.P(f"VND {money_in_finhay():,}")
                        ], style={'display':'inline-block', 
                                  "height": "30px", 
                                  "text-align": "center",
                                  "float":"right",
                                 }),
                        
                        html.Hr(
                            className="dashed"
                        ),
                        
                        html.Div([
                            html.H5("Gold")
                        ]),
                        
                    ], style={"width":"200px",
                              "height":"80px",
                              "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)", 
                              "border-radius": "5px",
                              "padding": "20px 20px",
                              'display':'inline-block',
                              "margin-right": "20px",
                              "margin-bottom": "20px"
                              }),
                    
                    # Macbook savings
                    html.Div([
                        # image
                        html.Div([
                            html.Img(
                                src="/assets/finhay.png",
                                height=30
                            )
                        ], style={'display':'inline-block',
                                  "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)",
                                  "border-radius": "5px 5px 5px 5px",
                                  "height": "30px"}),
                        
                        # amount
                        html.Div([
                            html.P(f"VND {money_in_finhay():,}")
                        ], style={'display':'inline-block', 
                                  "height": "30px", 
                                  "text-align": "center",
                                  "float":"right",
                                 }),
                        
                        html.Hr(
                            className="dashed"
                        ),
                        
                        html.Div([
                            html.H5("Macbook")
                        ]),
                        
                    ], style={"width":"200px",
                              "height":"130px",
                              "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)", 
                              "border-radius": "5px",
                              "padding": "20px 20px",
                              'display':'inline-block',
                              "margin-right": "20px",
                              "margin-bottom": "20px"
                              }),
                    
                    # Emergency savings
                    html.Div([
                        # image
                        html.Div([
                            html.Img(
                                src="/assets/finhay.png",
                                height=30
                            )
                        ], style={'display':'inline-block',
                                  "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)",
                                  "border-radius": "5px 5px 5px 5px",
                                  "height": "30px"}),
                        
                        # amount
                        html.Div([
                            html.P(f"VND {money_in_finhay():,}")
                        ], style={'display':'inline-block', 
                                  "height": "30px", 
                                  "text-align": "center",
                                  "float":"right",
                                 }),
                        
                        html.Hr(
                            className="dashed"
                        ),
                        
                        html.Div([
                            html.H5("Emergency")
                        ]),
                        
                    ], style={"width":"200px",
                              "height":"80px",
                              "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)", 
                              "border-radius": "5px",
                              "padding": "20px 20px",
                              'display':'inline-block',
                              "margin-right": "20px",
                              "margin-bottom": "20px"
                              }),
                    
                    # Donation savings
                    html.Div([
                        # image
                        html.Div([
                            html.Img(
                                src="/assets/finhay.png",
                                height=30
                            )
                        ], style={'display':'inline-block',
                                  "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)",
                                  "border-radius": "5px 5px 5px 5px",
                                  "height": "30px"}),
                        
                        # amount
                        html.Div([
                            html.P(f"VND {money_in_finhay():,}")
                        ], style={'display':'inline-block', 
                                  "height": "30px", 
                                  "text-align": "center",
                                  "float":"right",
                                 }),
                        
                        html.Hr(
                            className="dashed"
                        ),
                        
                        html.Div([
                            html.H5("Donation")
                        ]),
                        
                    ], style={"width":"200px",
                              "height":"80px",
                              "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)", 
                              "border-radius": "5px",
                              "padding": "20px 20px",
                              'display':'inline-block',
                              "margin-right": "20px",
                              "margin-bottom": "20px"
                              }),
                    
                    html.Hr(className="dashed")
                ])
            ]),
            
            dcc.Tab(id='tab', value='tab', label="Entry", children=[
                    
                html.H1("Add New Entry"),
                dash_table.DataTable(
                            id='entry-db-table',
                            data=df[display_columns].iloc[::-1].to_dict('records'),
                            columns=[{"name": i, "id": i} for i in display_columns],
                            editable=True,
                            page_size=10,
#                             fixed_rows={'headers': True},
#                             style_table={'height': 400},
                            style_cell={
                                        'minWidth': 80, 'maxWidth': 250, 'width': 80
                            }
                       ),
                                
                html.Br(),
                
                dcc.RadioItems(
                    id='entry-radio-button',
                    options=[
                        {'label': 'Add New Entry', 'value': 'add'},
                        {'label': 'Remove Entry', 'value': 'remove'}
                    ],
                    value='add',
                    labelStyle={'display': 'inline-block'}
                ),
                
                html.Br(),
                # ---
                
                html.Div(id='entry-option-display'),
                # ---
                   
            ]),
            
            
            
            dcc.Tab(label='More Information', children=[
                
                html.Div([
                    dcc.Markdown(info_content)
                ])

            ])
        ])
            ,
    
    ])

])

# category pie chart
@app.callback(
    Output("category-pie", "figure"), 
    [Input("start-date-pie", "value"), 
     Input("start-month-pie", "value"),
     Input("start-year-pie", "value"), 
     Input("end-date-pie", "value"),
     Input("end-month-pie", "value"), 
     Input("end-year-pie", "value")]
)
def display_category_pie_chart(start_date, start_month, start_year,
                               end_date, end_month, end_year):

    return create_category_pie_chart(df, f"{start_year}-{start_month}-{start_date}", 
                                         f"{end_year}-{end_month}-{end_date}")

# cumulative expenses
@app.callback(
    Output("cumulative_monthly_expenses", "figure"), 
    [Input("month_cumulative", "value"), Input("year_cumulative", "value")]
)
def display_cumulative_monthly_expenses(month_cumulative, year_cumulative):
    
    return create_cumulative_monthly_expenses_plot(df, year_cumulative, month_cumulative)

# daily expenses
@app.callback(
    Output("daily_expenses", "figure"), 
    [Input("month_daily", "value"), Input("year_daily", "value")]
)
def display_daily_expenses(month_daily, year_daily):
    
    return create_daily_expenses_plot(df, year_daily, month_daily)

# update category list when transaction_type changes
@app.callback(
    Output("transaction-category-filter", "options"), 
    [Input("transaction-type-filter", "value")]
)
def create_list_for_transaction_category(transaction_type):
    if transaction_type == 'All':
        transaction_category = df['category'].unique().tolist()
        transaction_category = ['All'] + transaction_category
    else:
        transaction_category = df[df['transaction_type'] == transaction_type]['category'].unique().tolist()
        transaction_category = ['All'] + transaction_category
    
    return [{"label": i, "value": i} for i in transaction_category]

# update sub-category list when category changes
@app.callback(
    Output("transaction-sub-category-filter", "options"), 
    [Input("transaction-type-filter", "value"),
     Input("transaction-category-filter", "value")]
)
def create_list_for_transaction_sub_category(transaction_type, category):
    if transaction_type == 'All':
        transaction_sub_category = df['sub_category'].unique().tolist()
        transaction_sub_category = ['All'] + transaction_sub_category
        
    elif category == 'All':
        transaction_sub_category = df[(df['transaction_type'] == transaction_type)]['sub_category'].unique().tolist()
        transaction_sub_category = ['All'] + transaction_sub_category
    else:
        transaction_sub_category = df[(df['transaction_type'] == transaction_type) & ((df['category'] == category))]['sub_category'].unique().tolist()
        transaction_sub_category = ['All'] + transaction_sub_category
    
    return [{"label": i, "value": i} for i in transaction_sub_category]


# default All when new option
@app.callback(
    Output("transaction-category-filter", "value"), 
    [Input("transaction-type-filter", "value")]
)
def default_all_category(transaction_type):
    return "All"

# reset transaction_type when update category
# UPDATE: wont do because will create dependency cycle: 
# transaction-type-filter.value -> transaction-category-filter.value -> transaction-type-filter.value

# default All when new option
@app.callback(
    Output("transaction-sub-category-filter", "value"), 
    [Input("transaction-type-filter", "value"),
     Input("transaction-category-filter", "value")]
)
def default_all_sub_category(transaction_type, category):
    return "All"

# transaction history
@app.callback(
    Output("transaction-history", "data"), 
    [Input("start-date-transaction", "value"), 
     Input("start-month-transaction", "value"),
     Input("start-year-transaction", "value"), 
     Input("end-date-transaction", "value"),
     Input("end-month-transaction", "value"), 
     Input("end-year-transaction", "value"),
     Input("transaction-type-filter", "value"),
     Input("transaction-category-filter", "value"),
     Input("transaction-sub-category-filter", "value")]
)
def display_daily_expenses(start_date, start_month, start_year,
                           end_date, end_month, end_year,
                           transaction_type,
                           category,
                           sub_category):
    
    transaction_df = df.copy()
    transaction_df['date'] = pd.to_datetime(transaction_df['date'])
    start_day = f"{start_year}-{start_month}-{start_date}"
    end_day = f"{end_year}-{end_month}-{end_date}"

    transaction_df = transaction_df[(transaction_df['date'] >= start_day) & (transaction_df['date'] <= end_day)]
    
    transaction_df['date'] = transaction_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    if transaction_type != "All":
        transaction_df = transaction_df[transaction_df['transaction_type'] == transaction_type]
        
    if category != "All":
        transaction_df = transaction_df[transaction_df['category'] == category]
        
    if sub_category != "All":
        transaction_df = transaction_df[transaction_df['sub_category'] == sub_category]
        
    transaction_df = transaction_df.append(transaction_df.sum(numeric_only=True), ignore_index=True)
    
    transaction_df.loc[len(transaction_df)-1, 'date'] = 'Sum'
    
    # cast from float to int to format
    transaction_df['amount'] = transaction_df['amount'].astype('int64')
    transaction_df['amount'] = transaction_df['amount'].map('{:,d}'.format)
    return transaction_df.to_dict('records')

# add a new entry row to preview table
@app.callback(
    Output('entry-preview-table-add', 'data'),
    Input('entry-enter', 'n_clicks'),
    State('entry-preview-table-add', 'data'),
    State('entry-preview-table-add', 'columns'),
    State('entry-date', 'date'),
    State('entry-transaction-type', 'value'),
    State('entry-category', 'value'),
    State('entry-sub-category', 'value'),
    State('entry-amount', 'value'),
    State('entry-currency', 'value'),
    State('entry-note', 'value'),
    State('entry-in-out', 'value'))
def add_row_to_preview_table(n_clicks, rows, columns,
            date, transaction_type, category, sub_category, amount, currency, note, in_out):
    sign = ''
    if in_out == "Cash Out":
        sign = '-'
        
    date = date.split('T')[0]
    [year, month, day] = map(int, date.split('-'))
    weekday = datetime.date(year, month, day).strftime('%A')
    
    if note == None:
        note = ''
    
    if n_clicks > 0:
        
        new_entry_df = pd.DataFrame({
                            'date' : f"{date}",
                            'date_of_week' : f"{weekday}",
                            'currency' : f"{currency}",
                            'transaction_type' : f"{transaction_type}",
                            'category' : f"{category}",
                            'sub_category' : f"{sub_category}",
                            'amount' : f"{sign}{amount}",
                            'note' : f"{note}"
                        }, index=[0])
        
        if rows == None:
            sum_df = new_entry_df
        else: 
            sum_df = pd.DataFrame.from_records(rows).iloc[:-1]
            sum_df = sum_df.append(new_entry_df)
            
        sum_df['amount'] = sum_df['amount'].astype('int64')
        sum_df = sum_df.append(sum_df.sum(numeric_only=True), ignore_index=True)
    
        sum_df.loc[len(sum_df)-1, 'date'] = 'Sum'
        
        return sum_df.to_dict("records")

# save to database add
@app.callback(
    Output('entry-db-table', 'data'),
    Input('entry-save-add', 'n_clicks'),
    State('entry-preview-table-add', 'data'),
    
    Input('entry-save-remove', 'n_clicks'),
    State('entry-preview-table-remove', 'data'),
    State("entry-start-date-remove", "value"), 
     State("entry-start-month-remove", "value"),
     State("entry-start-year-remove", "value"), 
     State("entry-end-date-remove", "value"),
     State("entry-end-month-remove", "value"), 
     State("entry-end-year-remove", "value"))
    
def update_entry_table(n_clicks_add, rows_add,
                      n_clicks_remove, rows_remove, 
                       start_date, start_month, start_year,
                       end_date, end_month, end_year):
     
    if n_clicks_add > 0: 
        rows = rows_add
        new_entry_df = pd.DataFrame.from_records(rows).iloc[:-1]
        database_df = pd.read_csv(database_name)
        database_df = database_df.append(new_entry_df)
        
        database_df['Date'] = pd.to_datetime(database_df['date'])
        database_df.sort_values(by=['Date'], inplace=True)
        database_df.drop(columns = ["Date"], inplace=True)
        
        database_df.to_csv(database_name, index=False)
        
    if n_clicks_remove > 0: 
        rows = rows_remove
        old_entry = df.copy()
        old_entry['date'] = pd.to_datetime(old_entry['date'])
        start_day = f"{start_year}-{start_month}-{start_date}"
        end_day = f"{end_year}-{end_month}-{end_date}"

        old_entry = old_entry[(old_entry['date'] >= start_day) & 
                                              (old_entry['date'] <= end_day)]

        old_entry['date'] = old_entry['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    
        
        new_entry = pd.DataFrame.from_records(rows).iloc[:-1]
        
        
        database_df = pd.read_csv(database_name)
        database_df = pd.concat([old_entry, database_df]).drop_duplicates(keep=False)

        database_df = database_df.append(new_entry)
        
        database_df['Date'] = pd.to_datetime(database_df['date'])
        database_df.sort_values(by=['Date'], inplace=True)
        database_df.drop(columns = ["Date"], inplace=True)
        
        database_df.to_csv(database_name, index=False)
    return database_df[display_columns].iloc[::-1].to_dict('records')


# refresh entry (amount) after saving
@app.callback(
    Output('entry-amount', 'value'),
    Input('entry-save-add', 'n_clicks'))
def add_row(n_clicks):
    if n_clicks > 0:
        return ''
    
# refresh entry (note) after saving
@app.callback(
    Output('entry-note', 'value'),
    Input('entry-save-add', 'n_clicks'))
def add_row(n_clicks):
    if n_clicks > 0:
        return ''
    
# update category list when transaction type changes
@app.callback(
    Output("entry-category", "options"), 
    [Input("entry-transaction-type", "value")]
)
def create_list_for_transaction_category(transaction_type):
    transaction_category = df[df['transaction_type'] == transaction_type]['category'].unique().tolist()
    
    return [{"label": i, "value": i} for i in transaction_category]

# update sub-category list when category changes
@app.callback(
    Output("entry-sub-category", "options"), 
    [Input("entry-transaction-type", "value"),
     Input("entry-category", "value")]
)
def create_list_for_transaction_sub_category(transaction_type, category):
    transaction_sub_category = df[(df['transaction_type'] == transaction_type) & ((df['category'] == category))]['sub_category'].unique().tolist()
    
    return [{"label": i, "value": i} for i in transaction_sub_category]

# enable/disable enter button before amount is filled
@app.callback(
    Output("entry-enter", "disabled"), 
    [Input("entry-sub-category", "value"),
     Input("entry-category", "value"),
     Input("entry-amount", "value")]
)
def enable_entry_enter_button(sub_category, category, amount):
    if (sub_category == None) or (category==None) or (amount==None):
        return True
    else:
        return False

# disable save to database button in add when there is entry
@app.callback(
    Output("entry-save-add", "disabled"), 
    [Input("entry-preview-table-add", "data")]
)
def enable_entry_enter_button(rows):
    if rows == None:
        return True
    else:
        return False

# display remove preview table and update sum when table changes
@app.callback(
    Output("entry-preview-table-remove", "data"), 
    [Input("entry-start-date-remove", "value"), 
     Input("entry-start-month-remove", "value"),
     Input("entry-start-year-remove", "value"), 
     Input("entry-end-date-remove", "value"),
     Input("entry-end-month-remove", "value"), 
     Input("entry-end-year-remove", "value"),
#      Input('entry-preview-table-remove', 'data_previous'),
#      State('entry-preview-table-remove', 'data')
    ]
)
def display_remove_preview_table(start_date, start_month, start_year,
                                 end_date, end_month, end_year,
#                                  data_previous, data
                                ):
      
    
    remove_preview_df = df.copy()
    remove_preview_df['date'] = pd.to_datetime(remove_preview_df['date'])
    start_day = f"{start_year}-{start_month}-{start_date}"
    end_day = f"{end_year}-{end_month}-{end_date}"

    remove_preview_df = remove_preview_df[(remove_preview_df['date'] >= start_day) & 
                                          (remove_preview_df['date'] <= end_day)]

    remove_preview_df['date'] = remove_preview_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    # update sum when table changes
#     if data_previous != data:
#         remove_preview_df = pd.DataFrame.from_records(data).iloc[:-1]
    
    
    remove_preview_df = remove_preview_df.append(remove_preview_df.sum(numeric_only=True), ignore_index=True)
    
    remove_preview_df.loc[len(remove_preview_df)-1, 'date'] = 'Sum'
    
    return remove_preview_df.to_dict('records')
    


# add entry radio button
@app.callback(
    Output("entry-option-display", "children"), 
    [Input("entry-radio-button", "value")]
)
def display_add_remove_panel(value):
    if value == 'add':
        return html.Div([
            html.Div([
                html.Div("On Date", 
                    style={'display':'block'}
                ),

                dcc.DatePickerSingle(
                        id='entry-date',
                        min_date_allowed=datetime.date(2001, 6, 4),
                        max_date_allowed=datetime.datetime.now(),
                        initial_visible_month=datetime.datetime.now(),
                        date=datetime.datetime.now(),
                        display_format='DD/MM/YYYY'
                ),
            ], style={'display':'inline-block', 'float':'left'}),

            html.Div([
                html.Div("Cash In/Out", 
                     style={
                        'display': 'block',
                     }),

                dcc.Dropdown(
                    id="entry-in-out",
                    options=[{"label": i, "value": i} for i in ['Cash In', 'Cash Out']],
                    value='Cash In',
                    clearable=False
                ),
            ], style={'width':'10%', 'display': 'inline-block', 'float':'left'}),

            html.Div([
                html.Div("Type", 
                     style={
                        'display': 'block',
                     }),

                dcc.Dropdown(
                    id="entry-transaction-type",
                    options=[{"label": i, "value": i} for i in df['transaction_type'].unique()],
                    value='Expenses',
                    clearable=False
                ),
            ], style={'width':'10%', 'display': 'inline-block', 'float':'left'}),

            html.Div([
                html.Div("Category", 
                     style={
                        'display': 'block',
                     }),

                dcc.Dropdown(
                    id="entry-category",
                    clearable=False
                ),
            ], style={'width': '10%', 'display': 'inline-block', 'float':'left'}),

            html.Div([
                html.Div("Sub-category", 
                     style={
                        'display': 'block',
                     }),

                dcc.Dropdown(
                    id="entry-sub-category",
                    clearable=False
                ),
            ], style={'width': '10%', 'display': 'inline-block', 'float':'left'}),

            html.Div([

                html.Div("Amount", 
                     style={
                        'display': 'block',
                     }),

                dcc.Input(
                    id="entry-amount",
                    type='number'
                ),
            ], style={'width': '10%', 'display': 'inline-block', 'float':'left', 
                      'margin-right':'50px',
                      'margin-left': '5px'}),

            html.Div([
                html.Div("Currency", 
                     style={
                        'display': 'block',
                     }),

                dcc.Dropdown(
                    id="entry-currency",
                    options=[{"label": i, "value": i} for i in df['currency'].unique()],
                    value=df['currency'].unique()[0],
                    clearable=False
                ),
            ], style={'width': '10%', 'display': 'inline-block', 'float':'left'}),

            html.Div([

                html.Div("Note", 
                     style={
                        'display': 'block',
                     }),

                dcc.Input(
                    id="entry-note",
                    type='text',
                ),
            ], style={'width': '10%', 'display': 'inline-block', 'margin-right':'50px'}),

            html.Div([
                html.Div(". ", 
                     style={
                        'display': 'block',
                     }),
                html.Button("Enter", 
                            id='entry-enter', 
                            n_clicks=0,
                            disabled = True)  
            ], style={'display':'inline-block', 'float':'right'}),


            html.Br(),
            dash_table.DataTable(
                    id='entry-preview-table-add',
#                             data=df[display_columns].iloc[::-1].to_dict('records'),
                    columns=[{"name": i, "id": i} for i in display_columns],
                    editable=True,
                    page_size=10,
                    style_cell={
                        'minWidth': 80, 'maxWidth': 250, 'width': 80
                    },
                    row_deletable=True,
               ),


            html.Br(),

            html.Button('Save to Database', 
                        id='entry-save-add', 
                        n_clicks=0, 
                        style={'display':'block'},
                        disabled=True),
            
            html.Br(),
            html.Br(),
            html.Br(),
        ])
            
    else:
        return html.Div([

            html.Div([
                html.Div("From", 
                         style={'width': '10%', 
                                'height':'50%',
                                'display': 'block',
                         }),

                html.Div([
                    dcc.Dropdown(
                        id="entry-start-date-remove",
                        options=[{"label": str(i).zfill(2), "value": str(i).zfill(2)} for i in range(1, 32)],
                        value=today_date.zfill(2),
                        clearable=False
                    ),
                ], style={'width': '20%', 'display': 'inline-block'}),

                html.Div([
                    dcc.Dropdown(
                        id="entry-start-month-remove",
                        options=[{"label": 'January'  , "value": '01'},
                                 {"label": 'February' , "value": '02'},
                                 {"label": 'March'    , "value": '03'},
                                 {"label": 'April'    , "value": '04'},
                                 {"label": 'May'      , "value": '05'},
                                 {"label": 'June'     , "value": '06'},
                                 {"label": 'July'     , "value": '07'},
                                 {"label": 'August'   , "value": '08'},
                                 {"label": 'September', "value": '09'},
                                 {"label": 'October'  , "value": '10'},
                                 {"label": 'November' , "value": '11'},
                                 {"label": 'December' , "value": '12'}],
                        value=today_month,
                        clearable=False
                    ),
                ], style={'width': '35%', 'display': 'inline-block'}),

                html.Div([
                    dcc.Dropdown(
                        id="entry-start-year-remove",
                        options=[{"label": i  , "value": i} for i in range(int(first_day_year), datetime.datetime.now().year+1)],
                        value=today_year,
                        clearable=False
                    ),
                ], style={'width': '20%', 'display': 'inline-block'})
            ], style={'width': '25%', 'display': 'inline-block'}),
            #----------

            html.Div([
                html.Div("To", 
                             style={'width': '10%', 
                                    'height':'50%',
                                    'display': 'block',
                             }),
                html.Div([
                    dcc.Dropdown(
                        id="entry-end-date-remove",
                        options=[{"label": str(i).zfill(2), "value": str(i).zfill(2)} for i in range(1, 32)],
                        value=today_date,
                        clearable=False
                    ),
                ], style={'width': '10%', 'display': 'inline-block'}),

                html.Div([
                    dcc.Dropdown(
                        id="entry-end-month-remove",
                        options=[{"label": 'January'  , "value": '01'},
                                 {"label": 'February' , "value": '02'},
                                 {"label": 'March'    , "value": '03'},
                                 {"label": 'April'    , "value": '04'},
                                 {"label": 'May'      , "value": '05'},
                                 {"label": 'June'     , "value": '06'},
                                 {"label": 'July'     , "value": '07'},
                                 {"label": 'August'   , "value": '08'},
                                 {"label": 'September', "value": '09'},
                                 {"label": 'October'  , "value": '10'},
                                 {"label": 'November' , "value": '11'},
                                 {"label": 'December' , "value": '12'}],
                        value=today_month,
                        clearable=False
                    ),
                ], style={'width': '18%', 'display': 'inline-block'}),

                html.Div([
                    dcc.Dropdown(
                        id="entry-end-year-remove",
                        options=[{"label": i  , "value": i} for i in range(int(first_day_year), datetime.datetime.now().year+1)],
                        value=first_day_year,
                        clearable=False
                    ),
                ], style={'width': '10%', 'display': 'inline-block'})
            ], style={'width': '50%', 'display': 'inline-block'}),


            html.Br(),
            dash_table.DataTable(
                    id='entry-preview-table-remove',
                    columns=[{"name": i, "id": i} for i in display_columns],
                    editable=True,
                    fixed_rows={'headers': True},
                    style_table={'height': 400},
                    style_cell={
                        'minWidth': 80, 'maxWidth': 250, 'width': 80
                    },
                    row_deletable=True,
               ),


            html.Br(),

            html.Button('Save to Database', id='entry-save-remove', n_clicks=0, style={'display':'block'}),
            
            html.Br(),
        ])
    

app.run_server(mode='external', debug=True)

Dash app running on http://127.0.0.1:8050/


In [28]:
# shutdown the server
# app._terminate_server_for_port("localhost", 8050)